# Youtube Downloader

In [3]:
ls = [
    "https://www.youtube.com/watch?v=GoM-64EkchU"
]

In [5]:
from pytube import YouTube

def Download(link):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    try:
        youtubeObject.download()
    except:
        print("An error has occurred")
    print("Download is completed successfully")

    
# for i in range(len(ls)):
#     link = ls[i]
#     Download(link)

# Downloading Transcipt From Youtube

In [16]:
from youtube_transcript_api import YouTubeTranscriptApi
video_id = "GoM-64EkchU"
subs =  YouTubeTranscriptApi.get_transcript(video_id)

In [21]:
subtitle = ""
for i in subs:
    subtitle += i['text']+" "

# Subtitles Generation

In [2]:
import pvleopard
access_key = "C6VWKOxIaasmIl4mjfnimk7MmKVwx7ygl6zuDDFXBgiM+DQUr7e4zA=="
leopard = pvleopard.create(access_key=access_key)

# Voice To Text

In [8]:
audio_path = "C:/Users/HP/Hack 7.0/vid1.mp4"

In [9]:
transcript, words = leopard.process_file(audio_path)

# Checking Accuracy

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

Document1 = subtitle
Document2 = transcript

corpus = [Document1,Document2]

X_train_counts = count_vect.fit_transform(corpus)

pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=['Document 1','Document 2'])

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

trsfm=vectorizer.fit_transform(corpus)
pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=['Document 1','Document 2'])


from sklearn.metrics.pairwise import cosine_similarity

cs = cosine_similarity(trsfm[0:1], trsfm)[0][1] * 100

print("Cosine Similarity : ",cs)

Cosine Similarity :  98.15122040495613


C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Converting Subtitles in srt Format

In [39]:
from typing import Sequence
from typing import Optional

def second_to_timecode(x: float) -> str:
    hour, x = divmod(x, 3600)
    minute, x = divmod(x, 60)
    second, x = divmod(x, 1)
    millisecond = int(x * 1000.)


    return '%.2d:%.2d:%.2d,%.3d' % (hour, minute, second, millisecond)


def to_srt(
        words: Sequence[pvleopard.Leopard.Word],
        endpoint_sec: float = 1.,
        length_limit: Optional[int] = 16) -> str:
    def _helper(end: int) -> None:
        lines.append("%d" % section)
        lines.append(
            "%s --> %s" %
            (
                second_to_timecode(words[start].start_sec),
                second_to_timecode(words[end].end_sec)
            )
        )
        lines.append(' '.join(x.word for x in words[start:(end + 1)]))
        lines.append('')


    lines = list()
    section = 0
    start = 0
    for k in range(1, len(words)):
        if ((words[k].start_sec - words[k - 1].end_sec) >= endpoint_sec) or \
                (length_limit is not None and (k - start) >= length_limit):
            _helper(k - 1)
            start = k
            section += 1
    _helper(len(words) - 1)


    return '\n'.join(lines)

# Storing srt file in the Corresponding Folder

In [44]:
import os
subtitle_path = r"C:/Users/HP/Hack 7.0/vid1.srt"
with open(subtitle_path, 'w') as f:
    f.write(to_srt(words))